In [8]:
import os
import datetime
from bs4 import BeautifulSoup
import requests
import re
from tqdm import tqdm
import json
import sqlite3
import pandas as pd

# Global headers definition
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

def makeUrlByDate(search, start_date, end_date):
    urls = []
    delta = end_date - start_date
    for i in range(delta.days + 1):
        date = start_date + datetime.timedelta(days=i)
        date_str = date.strftime("%Y%m%d")
        url = f"https://search.naver.com/search.naver?where=news&query={search}&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3&ds={date_str}&de={date_str}&docid=&nso=so%3Ar%2Cp%3Afrom{date_str}to{date_str}%2Ca%3A&mynews=0&refresh_start=0&related=0"
        urls.append(url)
    return urls

def articles_crawler(url):
    # html 불러오기
    original_html = requests.get(url, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")

    url_naver = html.select("div.group_news > ul.list_news > li div.news_area > div.news_info > div.info_group > a.info")
    article_urls = [u['href'] for u in url_naver]
    return article_urls

# 검색할 키워드 입력
search = input("검색할 키워드를 입력하세요: ")

# 크롤링할 시작 날짜 입력
start_date_str = input("\n크롤링할 시작 날짜를 입력하세요 (YYYY-MM-DD 형식): ")
start_date = datetime.datetime.strptime(start_date_str, "%Y-%m-%d")

# 크롤링할 종료 날짜 입력
end_date_str = input("\n크롤링할 종료 날짜를 입력하세요 (YYYY-MM-DD 형식): ")
end_date = datetime.datetime.strptime(end_date_str, "%Y-%m-%d")

# naver url 생성
urls = makeUrlByDate(search, start_date, end_date)

# 뉴스 크롤러 실행
news_titles = []
news_url = []
news_contents = []
news_dates = []
for i in urls:
    url = articles_crawler(i)
    news_url.append(url)

# 제목, 링크, 내용 1차원 리스트로 꺼내는 함수 생성
def makeList(newlist, content):
    for i in content:
        for j in i:
            newlist.append(j)
    return newlist

# 제목, 링크, 내용 담을 리스트 생성
news_url_1 = []

# 1차원 리스트로 만들기(내용 제외)
makeList(news_url_1, news_url)

# NAVER 뉴스만 남기기
final_urls = []
for i in tqdm(range(len(news_url_1))):
    if "news.naver.com" in news_url_1[i]:
        final_urls.append(news_url_1[i])
    else:
        pass

# 뉴스 내용 크롤링
for i in tqdm(final_urls):
    # 각 기사 html get하기
    news = requests.get(i, headers=headers)
    news_html = BeautifulSoup(news.text, "html.parser")

    # 뉴스 제목 가져오기
    title = news_html.select_one("#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    if title == None:
        title = news_html.select_one("#content > div.end_ct > div > h2")

    # 뉴스 본문 가져오기
    content = news_html.select("article#dic_area")
    if content == []:
        content = news_html.select("#articeBody")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=str(title))
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    news_titles.append(title)
    news_contents.append(content)

    try:
        html_date = news_html.select_one("div#ct> div.media_end_head.go_trans > div.media_end_head_info.nv_notrans > div.media_end_head_info_datestamp > div > span")
        news_date = html_date.attrs['data-date-time']
    except AttributeError:
        news_date = news_html.select_one("#content > div.end_ct > div > div.article_info > span > em")
        news_date = re.sub(pattern=pattern1, repl='', string=str(news_date))
    # 날짜 가져오기
    news_dates.append(news_date)

print("검색된 기사 갯수: 총 ", len(final_urls), '개')
print("\n[뉴스 제목]")
print(news_titles)
print("\n[뉴스 링크]")
print(final_urls)
print("\n[뉴스 내용]")
print(news_contents)

print('news_title: ', len(news_titles))
print('news_url: ', len(final_urls))
print('news_contents: ', len(news_contents))
print('news_dates: ', len(news_dates))

# 데이터 프레임으로 만들기
news_df = pd.DataFrame({'date': news_dates, 'title': news_titles, 'link': final_urls, 'content': news_contents})

# 중복 행 지우기
news_df = news_df.drop_duplicates(keep='first', ignore_index=True)
print("중복 제거 후 행 개수: ", len(news_df))

# 현재 시간 가져오기
now = datetime.datetime.now()

# 절대 경로로 JSON 파일 저장
output_path = os.path.join(os.getcwd(), '{}_{}.json'.format(search, now.strftime('%Y%m%d_%H시%M분%S초')))
with open(output_path, 'w', encoding='utf-8') as json_file:
    json_file.write(news_df.to_json(orient='records', force_ascii=False))

# 절대 경로로 데이터베이스 연결
db_path = os.path.join(os.getcwd(), 'news.db')
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 뉴스 데이터를 저장할 테이블 생성
cursor.execute('''CREATE TABLE IF NOT EXISTS news (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date TEXT,
                title TEXT,
                link TEXT,
                content TEXT
                )''')

# 데이터프레임의 데이터를 데이터베이스에 삽입
for index, row in news_df.iterrows():
    cursor.execute('''INSERT INTO news (date, title, link, content) VALUES (?, ?, ?, ?)''', (row['date'], row['title'], row['link'], row['content']))

# 변경 사항 커밋 및 연결 종료
conn.commit()
conn.close()


검색할 키워드를 입력하세요:  야구

크롤링할 시작 날짜를 입력하세요 (YYYY-MM-DD 형식):  2024-05-06

크롤링할 종료 날짜를 입력하세요 (YYYY-MM-DD 형식):  2024-05-07


100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.36s/it]

검색된 기사 갯수: 총  3 개

[뉴스 제목]
['고무팔→주먹야구 전도사 된 차명주…‘국민 놀이’ 찜뿌가 올림픽에?[이헌재의 인생홈런]', "역대 최고치 찍은 티빙...프로야구 유료화 '시험대'", '롯데 성적은 꼴찌… 가을야구 적금은 완판']

[뉴스 링크]
['https://n.news.naver.com/mnews/article/020/0003563033?sid=103', 'https://n.news.naver.com/mnews/article/014/0005181126?sid=105', 'https://n.news.naver.com/mnews/article/082/0001268640?sid=101']

[뉴스 내용]
['[\n\n\n\n\n차명주 대한야구소프트볼협회 이사 겸 베이스볼5 한국 대표팀 감독.    이헌재 기자 uni@donga.com고무공 하나, 테니스공 하나만 있으면 어디서나 할 수 있던 ‘주먹야구’를 기억하시는지. 동네에 따라 찜뿌, 찜뽕, 짬뽕, 손야구 등 다양한 이름으로 불렸던 주먹야구는 한때 모든 어린이들이 사랑했던 ‘국민 놀이’이자 운동이었다.   그런데 어쩌면 이 주먹야구가 올림픽 정식종목이 되는 날이 올지도 모른다. ‘베이스볼5’라는 멋진 이름을 단 이 종목은 한국에서는 이제 막 걸음마를 뗀 수준이지만 유럽과 남미, 아프리카 등에서는 이미 큰 인기를 얻고 있기 때문이다.   세계야구소프트볼연맹(WBSC)은 ‘야구의 국제화’와 어린 야구팬들을 겨냥해 2018년 베이스볼5의 공식 규칙을 만들었다. 그로부터 채 10년이 지나지 않아 국제올림픽위원회(IOC)는 세네갈에서 열리는 2026 다카르 청소년 올림픽에서 베이스볼5를 정식종목으로 채택했다. 추억의 주먹야구가 이제는 세계인이 즐기는 종목으로 발전한 것이다. \n\n\n\n차명주 감독을 비롯한 한국 베이스볼5 대표팀 선수단이 지난 달 서울에서 열린 아시아컵에서 관중들에게 인사를 하고 있다. 대한야구소프트볼협회 제공한국에도 엄연히 베이스볼5 국가대표가 있다. 고무공을 사용하는 이 종목